# core

> Fill in a module description here

In [ ]:
#| default_exp core

In [ ]:
#| hide
from nbdev.showdoc import *

In [ ]:
#| export
from bs4 import BeautifulSoup
from xml.dom import minidom
import re
import os

In [ ]:
#| export
class Client:
    def __init__(self, ndl_xml_path: str, title: str, output_path: str, manifest_path: str = None):
        self.manifest_path = manifest_path
        self.ndl_xml_path = ndl_xml_path
        self.title = title
        self.output_path = output_path

    def prettify(self, rough_string):
        reparsed = minidom.parseString(rough_string)
        pretty = re.sub(r"[\t ]+\n", "", reparsed.toprettyxml(indent="\t"))  # インデント後の不要な改行を削除
        pretty = pretty.replace(">\n\n\t<", ">\n\t<")  # 不要な空行を削除
        pretty = re.sub(r"\n\s*\n", "\n", pretty)  # 連続した改行（空白行を含む）を単一の改行に置換
        return pretty

    def convert(self):
        with open(self.ndl_xml_path, "r") as f:
            ndl_soup = BeautifulSoup(f, "xml")

        tei_str = f'''<?xml version="1.0" ?>
        <?xml-model href="http://www.tei-c.org/release/xml/tei/custom/schema/relaxng/tei_all.rng" type="application/xml" schematypens="http://relaxng.org/ns/structure/1.0"?>
        <?xml-model href="http://www.tei-c.org/release/xml/tei/custom/schema/relaxng/tei_all.rng" type="application/xml"
            schematypens="http://purl.oclc.org/dsdl/schematron"?>
        <!-- <?xml-stylesheet type="text/xsl" href="./mirador.xsl"?> -->
        <TEI xmlns="http://www.tei-c.org/ns/1.0">
            <teiHeader>
                <fileDesc>
                    <titleStmt>
                        <title>{self.title}</title>
                    </titleStmt>
                    <publicationStmt>
                        <p>Publication Information</p>
                    </publicationStmt>
                    <sourceDesc>
                        <p>Information about the source</p>
                    </sourceDesc>
                </fileDesc>
            </teiHeader>
            <text><body></body></text></TEI>'''

        tei_soup = BeautifulSoup(tei_str, "xml")

        ndl_pages = ndl_soup.find_all("PAGE")

        tei_body = tei_soup.find("body")

        # for i, canvas in enumerate(manifest["sequences"][0]["canvases"]): # [0:5]

        for i, ndl_page in enumerate(ndl_pages):  # [0:5]

            ## pb

            pb = tei_soup.new_tag("pb")
            pb["facs"] = "#p{}".format(i+1)
            pb["n"] = str(i+1)
            tei_body.append(pb)

            ## facsimile

            lines_map = {}

            for line in ndl_page.find_all("LINE"):
                order = int(line["ORDER"])

                if order not in lines_map:
                    lines_map[order] = line

            # for order in sorted(lines_map.keys()):
            for j, order in enumerate(sorted(lines_map.keys())):
                line = lines_map[order]

                ## ab

                ab = tei_soup.new_tag("ab")
                tei_body.append(ab)
                ab.string = line.attrs["STRING"]
                ab.attrs = {
                    "corresp": "#p{}-{}".format(i+1, j),
                    "type": line.attrs["TYPE"],
                }

                # line_tag = tei_soup.new_tag("zone")
                # line_tag.string = line_str
                # surface.append(line_tag)

                '''
                line_tag.attrs = {
                    "xml:id": "p{}-{}".format(i+1, j),
                    "ulx": line.attrs["X"],
                    "uly": line.attrs["Y"],
                    "lrx": str(int(line.attrs["X"]) + int(line.attrs["WIDTH"])),
                    "lry": str(int(line.attrs["Y"]) + int(line.attrs["HEIGHT"]))
                }
                '''

            '''

            surface = tei_soup.new_tag("surface")

            surface.attrs = {
                "sameAs": canvas["@id"],
                "xml:id": "p{}".format(i+1)
            }

            graphic = tei_soup.new_tag("graphic", url=canvas["images"][0]["resource"]["@id"])
            surface.append(graphic)
            facsimile.append(surface)

            ndl_page = ndl_pages[i]

            


            # for order in sorted(lines_map.keys()):
            for j, order in enumerate(sorted(lines_map.keys())):
                line = lines_map[order]

                ## ab

                ab = tei_soup.new_tag("ab")
                tei_body.append(ab)
                ab.string = line.attrs["STRING"]
                ab.attrs = {
                    "corresp": "#p{}-{}".format(i+1, j),
                    "type": line.attrs["TYPE"],
                }

                line_tag = tei_soup.new_tag("zone")
                # line_tag.string = line_str
                surface.append(line_tag)

                line_tag.attrs = {
                    "xml:id": "p{}-{}".format(i+1, j),
                    "ulx": line.attrs["X"],
                    "uly": line.attrs["Y"],
                    "lrx": str(int(line.attrs["X"]) + int(line.attrs["WIDTH"])),
                    "lry": str(int(line.attrs["Y"]) + int(line.attrs["HEIGHT"]))
                }
            '''

        '''
        if self.manifest_path:
            with open(self.manifest_path, "r") as f:
                manifest = json.load(f)

                facsimile = tei_soup.new_tag("facsimile")
                facsimile["sameAs"] = manifest["@id"]

                tei_soup.find("text").insert_after(facsimile)
        '''

        output_path = self.output_path
        os.makedirs(os.path.dirname(output_path), exist_ok=True)

        with open(output_path, "w") as f:
            f.write(self.prettify(str(tei_soup)))

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()